In [1]:
import numpy as np
import pandas as pd
df=pd.read_csv('../input/meld-dataset/MELD-RAW/MELD.Raw/train/train_sent_emo.csv', delimiter=',', nrows = None)

In [2]:
df.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my companys tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You mustve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So lets talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"


In [3]:
text_df=df['Utterance']
text_df=pd.DataFrame(text_df)
text_df.head()

,Utterance
0,also I was the point person on my companys tr...
1,You mustve had your hands full.
2,That I did. That I did.
3,So lets talk a little bit about your duties.
4,My duties? All right.


In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess(text):
    # Tokenize, pad, and truncate
    return tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')

text_df['input_ids'] = text_df['Utterance'].apply(lambda x: preprocess(x)['input_ids'][0])
text_df['attention_mask'] = text_df['Utterance'].apply(lambda x: preprocess(x)['attention_mask'][0])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self,input_ids,attention_masks):
        self.input_ids=input_ids
        self.attention_masks=attention_masks
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self,idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_masks[idx], dtype=torch.long),
        }

dataset=TextDataset(text_df['input_ids'].to_list(),text_df['attention_mask'].to_list())
train_loader = DataLoader(dataset, shuffle=False)

In [6]:
from transformers import BertModel

model = BertModel.from_pretrained('bert-base-uncased', num_labels=7).to('cuda')
model.eval()

from tqdm import tqdm
encodings=[]
with torch.no_grad():
        for batch in tqdm(train_loader):
            outputs = model(
                input_ids=batch['input_ids'].to('cuda'),
                attention_mask=batch['attention_mask'].to('cuda'),
            )
            encoding=outputs.last_hidden_state
            encodings.append(encoding)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

  0%|          | 0/9989 [00:00<?, ?it/s]/tmp/ipykernel_30/2523081496.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
/tmp/ipykernel_30/2523081496.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(self.attention_masks[idx], dtype=torch.long),
100%|██████████| 9989/9989 [01:36<00:00, 103.67it/s]


In [8]:
encodings[9987]

tensor([[[-0.3271, -0.0116, -0.0164,  ..., -0.1454,  0.4163,  0.1177],
         [ 0.7810, -0.1910,  0.3150,  ..., -0.3541,  0.9698, -0.2329],
         [-0.2891, -0.7291,  0.1984,  ...,  0.8826,  0.8325, -0.5647],
         ...,
         [ 0.2801,  0.3044,  0.5951,  ...,  0.1329,  0.4678, -0.0913],
         [-0.2380, -0.1202,  0.5665,  ...,  0.6372,  0.5232, -0.1867],
         [ 0.1859,  0.2201,  0.6492,  ...,  0.0793,  0.3894, -0.0810]]],
       device='cuda:0')

In [9]:
text_encodings=[t.flatten() for t in encodings]

In [12]:
bert_encodings_np = [encoding.cpu().numpy() for encoding in text_encodings]

In [14]:
type(bert_encodings_np[0])

numpy.ndarray

In [16]:
import pickle
with open('text_embeddings.pkl', 'wb') as f:
    pickle.dump(bert_encodings_np, f)

In [17]:
with open('text_embeddings.pkl', 'rb') as f:
    loaded_array_list = pickle.load(f)